In [1]:
# 1324124124
# 223345678
## testtest02060110pm
# test02061823pm

In [2]:
import requests
import pandas as pd
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import random
from plotly.offline import init_notebook_mode
from dateutil.relativedelta import relativedelta
init_notebook_mode(connected = True)
import warnings
warnings.filterwarnings('ignore')
import os
from scipy import stats
import json
import math
import base64
import hmac
import math
import base64
import struct
import hashlib
import time
from requests.auth import HTTPBasicAuth
import pandas as pd
from CONFIG import *
import ccxt
base_dir = get_base_dir()
pd.set_option('display.float_format', '{:.2f}'.format)  # 改成你想要的小数位数


proxies = {'https':'127.0.0.1:7890', 'http':'127.0.0.1:7890'}


def fetch_okx_last_price(base: str) -> float:
    """
    base: 'BTC' or 'ETH'
    优先拉永续:  BTC/USDT:USDT (OKX swap)
    回退拉现货:  BTC/USDT
    返回 ticker['last'] (float)
    """
    ex = ccxt.okx({
        "enableRateLimit": True,
        "options": {"defaultType": "swap"},  # 优先 swap
    })

    # 1) swap 优先
    try:
        sym_swap = f"{base}/USDT:USDT"
        t = ex.fetch_ticker(sym_swap)
        if t.get("last") is not None:
            return float(t["last"])
    except Exception:
        pass

    # 2) 回退 spot
    ex = ccxt.okx({
        "enableRateLimit": True,
        "options": {"defaultType": "spot"},
    })
    sym_spot = f"{base}/USDT"
    t = ex.fetch_ticker(sym_spot)
    return float(t["last"])
def cal_google_code(secret_key: str):
    # secret key 的长度必须是 8 的倍数。所以如果 secret key 不符合要求，需要在后面补上相应个数的 "="
    secret_key_len = len(secret_key)
    secret_key_pad_len = math.ceil(secret_key_len / 8) * 8 - secret_key_len
    secret_key = secret_key + "=" * secret_key_pad_len

    duration_input = int(time.time()) // 30
    _key = base64.b32decode(secret_key)
    msg = struct.pack(">Q", duration_input)
    google_code = hmac.new(_key, msg, hashlib.sha1).digest()
    o = google_code[19] & 15
    google_code = str((struct.unpack(">I", google_code[o:o+4])[0] & 0x7fffffff) % 1000000)

    # 生成的验证码未必是 6 位，注意要在前面补 0
    if len(google_code) == 5:
        google_code = '0' + google_code
    return google_code


def get_cookie(_url: str, _username: str, _password: str):
    _response = requests.post(url=_url, auth=HTTPBasicAuth(_username, _password), allow_redirects=False, stream=True)
    print(_response, _response.text, _response.headers)
    _cookies = requests.utils.dict_from_cookiejar(_response.cookies)
    return _cookies

# 返回一个被两重验证的、可以使用的cookie
def login(prefix="mmadminjp"):
    s = requests.Session()
    
    username1 = 'ray_xu' # dcdl用户名
    if prefix == 'mmadminjp3':
        key1 = 'NVCX4PJP32Y4JTJJVCKU5XUQ4M' # dcdl密钥
    elif prefix == 'mmadmin':
        key1 = '7H62KCRCUEOYH2FV4SCYXYMPWE'
    password1 = cal_google_code(key1)
#     print(username1, password1)
    # 如果用http://，会被重定向为https://，可以通过headers得到
    url = f'https://{prefix}.digifinex.org/'
    login_response1 = s.post(url=url, auth=HTTPBasicAuth(username1, password1), allow_redirects=False, stream=True)
#     print(login_response1, login_response1.text, login_response1.headers)
    login_cookies1 = requests.utils.dict_from_cookiejar(login_response1.cookies)
#     print(login_cookies1)
    username2 = 'ray_xu' # dcdl用户名
    key2 = 'HBRDG6BUPJZGC6K7PB2TI3LCOM4XQNDO' # dcdl密钥
    password2 = cal_google_code(key2)
    # 这里必须再登陆一次，否则会重定向到login
    url2 = f'https://{prefix}.digifinex.org/api/login/valid'
#     print(username2, password2, url)
    data = {
        "account":username2,
        "glecode":password2,
    }
    # 不加第一次登陆的cookie会报错
    login_response2 = s.post(url=url2, data = data, headers={'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36 Edg/120.0.0.0',
                                                                    'Content-Type': 'application/x-www-form-urlencoded'}, 
                                    cookies = login_cookies1, allow_redirects=False)
#     print(login_response2, login_response2.text, login_response2.headers)
    login_cookies2 = requests.utils.dict_from_cookiejar(login_response2.cookies)
    
    return s, login_cookies1

In [3]:
verfied_session, verfied_cookies = login(prefix='mmadmin')
verfied_session1, verfied_cookies1 = login(prefix='mmadminjp3')

In [4]:
dc_name = pd.read_excel(f'{base_dir}/Obentech/CF环境账户对应关系.xlsx')
dc_name.dropna(subset=['Binance账户名','本金'],inplace=True)
dc_name = dc_name[dc_name['备注'].isna()]
dc_name.reset_index(drop=True, inplace=True)
# 新增：类型（OKX账户名包含 zy/ZY -> 内部，否则托管）
# mask_internal = dc_name['OKX账户名'].fillna('').str.contains('zy', case=False, regex=False)
# dc_name['类型'] = np.where(mask_internal, '内部', '托管')



# 公司资金（内部）
env_company = ["cfdc_dcpro1","cfdc_dcpro2","cfdc_dcpro3","cfdc_dcpro4","cfdc_dcpro5","cfdc_dcpro7",
               "cfdc_dcpro8","cfdc_dcpro9","cfdc_dcpro12","cfdc_dcpro13","cfdc_dcpro15","cfdc_dcpro16",
               "cfdc_dcpro17","cfdc_dcpro24","cfdc_dcpro26","cfdc_dcpro27","cfdc_dcpro29"]

# 客户资金（托管）
env_client = ["cfdc_dcpro6","cfdc_dcpro10","cfdc_dcpro11","cfdc_dcpro14","cfdc_dcpro18","cfdc_dcpro19",
              "cfdc_dcpro20","cfdc_dcpro21","cfdc_dcpro25","cfdc_dcpro28","cfdc_dcpro30","cfdc_dcpro31",
              "cfdc_dcpro32","cfdc_dcpro33","cfdc_dcpro35","cfdc_dcpro38"]

type_map = {e: "内部" for e in env_company}
type_map.update({e: "托管" for e in env_client})

dc_name["类型"] = dc_name["跨所环境"].map(type_map).fillna("未知")



dc_name

,跨所环境,本金,Binance本金,OKX本金,Binance环境,Binance账户名,OKX环境,OKX账户名,备注,类型
0,cfdc_dcpro1,USDT,500000.00,500000.00,bn_v5_pm_pro5,binancetfmmtest50@dftstat.com,okx_v5_test12,Okzyusdt02,NaN,内部
1,cfdc_dcpro2,USDT,1000000.00,1000000.00,bn_v5_pm_pro6,gosfsrmanual2@dftstat.com,okx_v5_test36,OKZYUSDT03,NaN,内部
2,cfdc_dcpro3,USDT,1650000.00,2200000.00,bn_v5_pm_pro8,boshu_bntest8@dftstat.com,okx_v5_test37,okzyusdt04,NaN,内部
3,cfdc_dcpro4,USDT,1200000.00,1500000.00,bn_v5_pm_pro16,gOKXSFSR2@dftstat.com,okx_v5_test38,OKZYUSDT05,NaN,内部
4,cfdc_dcpro5,USDT,253380.00,289520.00,bn_v5_pm_pro20,boshu_bntest13@dftstat.com,okx_v5_test39,okcfcf01,NaN,内部
5,cfdc_dcpro6,BTC,10.00,13.51,bn_v5_pm_pro21,cf04ran20250214_virtual@ewmiw03nmanagedsub.com,okx_v5_test32,cf03ran20241230,NaN,托管
6,cfdc_dcpro7,USDT,600000.00,1750000.00,bn_v5_pm_pro22,bn61btc@dftstat.com,okx_v5_test41,okzyusdt06,NaN,内部
7,cfdc_dcpro8,USDT,670000.00,1366511.00,bn_v5_pm_pro1,binancetfmmtest51@dftstat.com,okx_v5_test42,okzyusdt07,NaN,内部
8,cfdc_dcpro9,USDT,939536.35,1579219.47,bn_v5_pm_pro24,boshu_bncf01@dftstat.com,okx_v5_test43,okzyusdt08,NaN,内部
9,cfdc_dcpro10,USDT,1409420.00,1669420.00,bn_v5_pm_pro15,cf09ran20250423_virtual@n9nik8gxmanagedsub.com,okx_v5_test44,cf09ran20250423okx,NaN,托管


In [5]:
env_info1 = pd.DataFrame(columns=['control', 'accountEquity', 'actualEquity','accountMaintMargin','quoteAmount', 'uniMMR'])
for j in  range(len(dc_name)):
    control = dc_name.loc[j, 'OKX环境']
    print(control)
    symbol = 'BTCUSDT'
    url = f"https://mmadmin.digifinex.org/api/cf_arbitrage/{symbol}"
    querystring = {"env":control}
    json = verfied_session.request("GET", url, cookies = verfied_cookies, params=querystring).json()
    if  json['data'] != None:
        data =json['data']
        account = data['detail']['account']
        accountEquity = float(account['accountEquity'])
        actualEquity = float(account['actualEquity'])
        quoteAmount = float(account['quoteAmount'])
        accountInitialMargin  = float(account['accountInitialMargin'])
        accountMaintMargin= float(account['accountMaintMargin'])
        rest_margin = actualEquity - accountInitialMargin
        uniMMR = float(account['uniMMR'])
    env_info1.loc[env_info1.shape[0]] = [control, accountEquity,actualEquity,accountMaintMargin, quoteAmount, uniMMR]
env_info1=env_info1[['control', 'accountEquity','accountMaintMargin', 'uniMMR']]
env_info1.columns=['OKX环境', 'okx_accountEquity', 'okx_accountMaintMargin', 'okx_uniMMR']
env_info1

okx_v5_test12
okx_v5_test36
okx_v5_test37
okx_v5_test38
okx_v5_test39
okx_v5_test32
okx_v5_test41
okx_v5_test42
okx_v5_test43
okx_v5_test44
okx_v5_test45
okx_v5_test2
okx_v5_test7
okx_v5_test46
okx_v5_test47
okx_v5_test48
okx_v5_test49
okx_v5_test10
okx_v5_test6
okx_v5_test11
okx_v5_test20
okx_v5_test22
okx_v5_test31
okx_v5_test5
okx_v5_test23
okex5_v4_maker_cross_test8
okx_v5_test28
okx_v5_test51
okx_v5_test52
okx_v5_test33
okx_v5_test9
okx_v5_test19
okx_v5_test53
okx_v5_test13
okx_v5_test55


,OKX环境,okx_accountEquity,okx_accountMaintMargin,okx_uniMMR
0,okx_v5_test12,394938.63,5027.50,76.18
1,okx_v5_test36,772292.50,33054.56,21.95
2,okx_v5_test37,1244973.90,61016.77,19.08
3,okx_v5_test38,1049116.51,70120.41,12.18
4,okx_v5_test39,204665.78,2965.45,64.01
5,okx_v5_test32,600874.10,23666.21,23.58
6,okx_v5_test41,700481.80,32492.97,20.76
7,okx_v5_test42,861887.53,24434.96,33.93
8,okx_v5_test43,870009.36,24906.69,34.03
9,okx_v5_test44,547547.34,20264.49,26.24


In [6]:
# verfied_session1, verfied_cookies1 = login(prefix='mmadminjp3')
env_info2 = pd.DataFrame(columns=['control', 'accountEquity', 'actualEquity','accountMaintMargin','quoteAmount', 'uniMMR'])
for j in  range(len(dc_name)):
    control = dc_name.loc[j, 'Binance环境']
    print(control)
    symbol = 'BTCUSDT'
    url = f"https://mmadminjp3.digifinex.org/api/cf_arbitrage/{symbol}"
    querystring = {"env":control}
    json = verfied_session1.request("GET", url, cookies = verfied_cookies1, params=querystring).json()
    if  json['data'] != None:
        data =json['data']
        account = data['detail']['account']
        accountEquity = float(account['accountEquity'])
        actualEquity = float(account['actualEquity'])
        quoteAmount = float(account['quoteAmount'])
        accountInitialMargin  = float(account['accountInitialMargin'])
        accountMaintMargin= float(account['accountMaintMargin'])
        rest_margin = actualEquity - accountInitialMargin
        uniMMR = float(account['uniMMR'])
    env_info2.loc[env_info2.shape[0]] = [control, accountEquity,actualEquity,accountMaintMargin, quoteAmount, uniMMR]
env_info2=env_info2[['control', 'accountEquity','accountMaintMargin', 'uniMMR']]
env_info2.columns=['Binance环境', 'binance_accountEquity', 'binance_accountMaintMargin', 'binance_uniMMR']
env_info2

bn_v5_pm_pro5
bn_v5_pm_pro6
bn_v5_pm_pro8
bn_v5_pm_pro16
bn_v5_pm_pro20
bn_v5_pm_pro21
bn_v5_pm_pro22
bn_v5_pm_pro1
bn_v5_pm_pro24
bn_v5_pm_pro15
bn_v5_pm_pro7
bn_v5_pm_pro13
bn_v5_pm_pro9
bn_v5_pm_pro17
bn_v5_pm_pro3
bn_v5_pm_pro4
bn_v5_pm_pro25
bn_v5_pm_pro18
bn_v5_pm_pro2
bn_v5_pm_pro28
bn_v5_pm_pro29
bn_v5_pm_pro19
bn_v5_pm_pro34
bn_v5_pm_pro35
bn_v5_pm_pro36
bn_v5_pm_pro37
bn_v5_pm_pro38
bn_v5_pm_pro14
bn_v5_pm_pro39
bn_v5_pm_pro40
bn_v5_pm_pro41
bn_v5_pm_pro42
bn_v5_pm_pro44
bn_v5_pm_pro45
bn_v5_pm_pro46


,Binance环境,binance_accountEquity,binance_accountMaintMargin,binance_uniMMR
0,bn_v5_pm_pro5,615134.40,10246.33,60.03
1,bn_v5_pm_pro6,1236818.11,29597.17,41.79
2,bn_v5_pm_pro8,1825525.00,58582.99,31.16
3,bn_v5_pm_pro16,1656204.81,55059.78,30.08
4,bn_v5_pm_pro20,341861.92,5531.36,61.80
5,bn_v5_pm_pro21,1023610.65,17275.53,59.25
6,bn_v5_pm_pro22,1365096.95,39389.23,34.66
7,bn_v5_pm_pro1,1197810.81,38023.14,31.50
8,bn_v5_pm_pro24,1464243.97,32137.22,45.56
9,bn_v5_pm_pro15,856074.57,35548.32,24.08


In [7]:
all_data = pd.merge(dc_name, env_info1, how='left', left_on='OKX环境',  right_on='OKX环境')
all_data = pd.merge(all_data, env_info2, how='left', left_on='Binance环境',  right_on='Binance环境')
all_data['total_equity'] = all_data['okx_accountEquity'] + all_data['binance_accountEquity']
all_data['mean'] = all_data['total_equity']/2
all_data['transfer_equity'] = all_data['okx_accountEquity']-all_data['mean']
all_data['pre_uniMMR'] = all_data['mean'] /all_data['okx_accountMaintMargin'] 
all_data['diff_ratio'] = (all_data['okx_accountEquity'] -all_data['mean'] ) /all_data['mean']
all_data['okx_mark'] = all_data['OKX环境'].apply(lambda x: x.split('_')[-1])
show_column = ['跨所环境', '本金', 'Binance本金', 'OKX本金', 'Binance环境', 'Binance账户名', 'OKX环境',
       'OKX账户名', '类型', 'okx_accountEquity',
       'okx_uniMMR', 'binance_accountEquity',
       'binance_uniMMR', 'total_equity', 'transfer_equity','diff_ratio']
all_data = all_data[show_column]
btc_okx = fetch_okx_last_price("BTC")
eth_okx = fetch_okx_last_price("ETH")

all_data["OKX_BTCUSDT_Price"] = btc_okx
all_data["OKX_ETHUSDT_Price"] = eth_okx

all_data["final_transfer"] = np.select(
    [
        all_data["本金"].eq("BTC"),
        all_data["本金"].eq("ETH"),
        all_data["本金"].eq("USDT"),
    ],
    [
        all_data["transfer_equity"] / all_data["OKX_BTCUSDT_Price"],
        all_data["transfer_equity"] / all_data["OKX_ETHUSDT_Price"],
        all_data["transfer_equity"],
    ],
    default=np.nan,  # 其他币种先留空
)

all_data['abs_diff_ratio'] = all_data['diff_ratio'].abs()
all_data.sort_values('abs_diff_ratio',ascending=False)

,跨所环境,本金,Binance本金,OKX本金,Binance环境,Binance账户名,OKX环境,OKX账户名,类型,okx_accountEquity,okx_uniMMR,binance_accountEquity,binance_uniMMR,total_equity,transfer_equity,diff_ratio,OKX_BTCUSDT_Price,OKX_ETHUSDT_Price,final_transfer,abs_diff_ratio
26,cfdc_dcpro29,ETH,120.00,181.06,bn_v5_pm_pro38,BINANCETFMMTEST9@dftstat.com,okx_v5_test28,okzyeth04,内部,279043.55,29.65,619528.43,215.11,898571.98,-170242.44,-0.38,69285.90,2083.01,-81.73,0.38
6,cfdc_dcpro7,USDT,600000.00,1750000.00,bn_v5_pm_pro22,bn61btc@dftstat.com,okx_v5_test41,okzyusdt06,内部,700481.80,20.76,1365096.95,34.66,2065578.75,-332307.58,-0.32,69285.90,2083.01,-332307.58,0.32
21,cfdc_dcpro24,BTC,8.00,34.96,bn_v5_pm_pro19,binancetfmmtest35@dftstat.com,okx_v5_test22,okzybtc03,内部,1052960.08,21.98,1906639.86,37.41,2959599.94,-426839.89,-0.29,69285.90,2083.01,-6.16,0.29
15,cfdc_dcpro16,USDT,1697287.00,1600000.00,bn_v5_pm_pro4,boshu_bntest20@dftstat.com,okx_v5_test48,okzyusdt12,内部,1091069.37,20.07,1918427.80,30.31,3009497.18,-413679.22,-0.27,69285.90,2083.01,-413679.22,0.27
17,cfdc_dcpro18,ETH,250.00,391.09,bn_v5_pm_pro18,digifinex_virtual@vzorp3xnmanag,okx_v5_test10,eth231206,托管,493084.40,18.44,850855.38,46.03,1343939.78,-178885.49,-0.27,69285.90,2083.01,-85.88,0.27
5,cfdc_dcpro6,BTC,10.00,13.51,bn_v5_pm_pro21,cf04ran20250214_virtual@ewmiw03nmanagedsub.com,okx_v5_test32,cf03ran20241230,托管,600874.10,23.58,1023610.65,59.25,1624484.76,-211368.28,-0.26,69285.90,2083.01,-3.05,0.26
8,cfdc_dcpro9,USDT,939536.35,1579219.47,bn_v5_pm_pro24,boshu_bncf01@dftstat.com,okx_v5_test43,okzyusdt08,内部,870009.36,34.03,1464243.97,45.56,2334253.32,-297117.31,-0.25,69285.90,2083.01,-297117.31,0.25
4,cfdc_dcpro5,USDT,253380.00,289520.00,bn_v5_pm_pro20,boshu_bntest13@dftstat.com,okx_v5_test39,okcfcf01,内部,204665.78,64.01,341861.92,61.80,546527.70,-68598.07,-0.25,69285.90,2083.01,-68598.07,0.25
16,cfdc_dcpro17,USDT,1689820.00,1618473.00,bn_v5_pm_pro25,binancetfmmtest21@dftstat.com,okx_v5_test49,okzyusdt13,内部,1181293.14,27.54,1969231.74,32.04,3150524.88,-393969.30,-0.25,69285.90,2083.01,-393969.30,0.25
32,cfdc_dcpro35,BTC,6.00,6.00,bn_v5_pm_pro44,bndellecodbtc5012026_virtual@g5pxufupmanagedsu...,okx_v5_test53,DellecodBTC05012026,托管,584282.49,29.06,940894.57,88.95,1525177.06,-178306.04,-0.23,69285.90,2083.01,-2.57,0.23


In [8]:
import numpy as np
import pandas as pd

imbalance_threshold = 0.25

# ===== 仅用于打印的数量格式：取绝对值；USDT -> xx万，BTC/ETH -> 0.01 =====
ft_abs = all_data["final_transfer"].astype(float).abs()

final_transfer_str = np.where(
    all_data["本金"].eq("USDT"),
    (ft_abs / 10_000).round(0).astype("Int64").astype(str) + "万",
    ft_abs.round(2).map(lambda x: f"{x:.2f}")
)
final_transfer_str = pd.Series(final_transfer_str, index=all_data.index)

# USDT 不加“个”
unit = pd.Series(np.where(all_data["本金"].eq("USDT"), "", "个"), index=all_data.index)

# ===== 过滤：托管 + |diff_ratio|>0.1 =====
need = (all_data["类型"].eq("托管")) & (all_data["diff_ratio"].abs() >imbalance_threshold)

# ===== 关键：按 |diff_ratio| 从大到小（你没有 abs_ratio，就用这个）=====
idx_sorted = all_data.loc[need].assign(abs_ratio=all_data.loc[need, "diff_ratio"].abs()) \
    .sort_values("abs_ratio", ascending=False).index

# ===== BN -> OKX (diff_ratio < 0)，保持 |diff_ratio| 降序 =====
idx_bn_to_okx = idx_sorted[all_data.loc[idx_sorted, "diff_ratio"] < 0]
lines_bn_to_okx = (
    "从BN账户" + all_data.loc[idx_bn_to_okx, "Binance账户名"].astype(str)
    + "转 " + final_transfer_str.loc[idx_bn_to_okx] + unit.loc[idx_bn_to_okx] + " " + all_data.loc[idx_bn_to_okx, "本金"].astype(str)
    + "到OKX账户" + all_data.loc[idx_bn_to_okx, "OKX账户名"].astype(str)
    + " （跨所环境为" + all_data.loc[idx_bn_to_okx, "跨所环境"].astype(str)
    + "，BN环境为" + all_data.loc[idx_bn_to_okx, "Binance环境"].astype(str)
    + "，OKX环境为" + all_data.loc[idx_bn_to_okx, "OKX环境"].astype(str) + "）"
).tolist()

# ===== OKX -> BN (diff_ratio > 0)，保持 |diff_ratio| 降序 =====
idx_okx_to_bn = idx_sorted[all_data.loc[idx_sorted, "diff_ratio"] > 0]
lines_okx_to_bn = (
    "从OKX账户" + all_data.loc[idx_okx_to_bn, "OKX账户名"].astype(str)
    + "转 " + final_transfer_str.loc[idx_okx_to_bn] + unit.loc[idx_okx_to_bn] + " " + all_data.loc[idx_okx_to_bn, "本金"].astype(str)
    + "到BN账户" + all_data.loc[idx_okx_to_bn, "Binance账户名"].astype(str)
    + " （跨所环境为" + all_data.loc[idx_okx_to_bn, "跨所环境"].astype(str)
    + "，BN环境为" + all_data.loc[idx_okx_to_bn, "Binance环境"].astype(str)
    + "，OKX环境为" + all_data.loc[idx_okx_to_bn, "OKX环境"].astype(str) + "）"
).tolist()

# ===== 分组打印：每行空一行 =====
print("BN到OKX")
for s in lines_bn_to_okx:
    print(s)
    print()

print("OKX到BN")
for s in lines_okx_to_bn:
    print(s)
    print()


BN到OKX
从BN账户digifinex_virtual@vzorp3xnmanag转 85.88个 ETH到OKX账户eth231206 （跨所环境为cfdc_dcpro18，BN环境为bn_v5_pm_pro18，OKX环境为okx_v5_test10）

从BN账户cf04ran20250214_virtual@ewmiw03nmanagedsub.com转 3.05个 BTC到OKX账户cf03ran20241230 （跨所环境为cfdc_dcpro6，BN环境为bn_v5_pm_pro21，OKX环境为okx_v5_test32）

OKX到BN


In [9]:
# ===== 过滤：内部 + |diff_ratio|>0.1 =====

imbalance_threshold = 0.25
need = (all_data["类型"].eq("内部")) & (all_data["diff_ratio"].abs() > imbalance_threshold)

# ===== 数量展示：取绝对值；USDT->万，BTC/ETH->0.01 =====
ft_abs = all_data["final_transfer"].astype(float).abs()

final_transfer_str = np.where(
    all_data["本金"].eq("USDT"),
    (ft_abs / 10_000).round(0).astype("Int64").astype(str) + "万",   # e.g. 144000 -> 14万
    ft_abs.round(2).map(lambda x: f"{x:.2f}")                      # BTC/ETH -> 0.01
)
final_transfer_str = pd.Series(final_transfer_str, index=all_data.index)

# ===== 两个方向 =====
m_okx_to_bn = need & (all_data["diff_ratio"] > 0)   # OKX -> BN
m_bn_to_okx = need & (all_data["diff_ratio"] < 0)   # BN -> OKX
env_pair_str = (
    "("
    + all_data["Binance环境"].astype(str)
    + " + "
    + all_data["OKX环境"].astype(str)
    + ")"
)
# ===== OKX -> BN =====
lines_okx_to_bn = (
    "从内部跨所账户组" + all_data.loc[m_okx_to_bn, "跨所环境"].astype(str)
    + env_pair_str.loc[m_okx_to_bn]
    + "的OKX账户" + all_data.loc[m_okx_to_bn, "OKX账户名"].astype(str)
    + "的funding账户提走 " + final_transfer_str.loc[m_okx_to_bn] + " " + all_data.loc[m_okx_to_bn, "本金"].astype(str)
    + ", 转入Binance账户" + all_data.loc[m_okx_to_bn, "Binance账户名"].astype(str)
    + "的现货账户中。"
).tolist()

# ===== BN -> OKX =====
lines_bn_to_okx = (
    "从内部跨所账户组" + all_data.loc[m_bn_to_okx, "跨所环境"].astype(str)
    + env_pair_str.loc[m_bn_to_okx]
    + "的Binance账户" + all_data.loc[m_bn_to_okx, "Binance账户名"].astype(str)
    + "的现货账户提走 " + final_transfer_str.loc[m_bn_to_okx] + " " + all_data.loc[m_bn_to_okx, "本金"].astype(str)
    + ", 转入OKX账户" + all_data.loc[m_bn_to_okx, "OKX账户名"].astype(str)
    + "的funding账户中。"
).tolist()

# ===== 打印整段申请模板 =====
print("Mark您好，因CF跨所套利策略账户资金不平衡，现申请在okx和binance之间转账：\n")

print("从OKX 转到 Binance：（如果有）")
if lines_okx_to_bn:
    for s in lines_okx_to_bn:
        print(s)
        print()
else:
    print("（无）\n")

print("从Binance转到 OKX：(如果有）")
if lines_bn_to_okx:
    for s in lines_bn_to_okx:
        print(s)
        print()
else:
    print("（无）\n")

print("\n资金已经都在两个交易所对应的资金账户了，谢谢Mark")


Mark您好，因CF跨所套利策略账户资金不平衡，现申请在okx和binance之间转账：

从OKX 转到 Binance：（如果有）
（无）

从Binance转到 OKX：(如果有）
从内部跨所账户组cfdc_dcpro5(bn_v5_pm_pro20 + okx_v5_test39)的Binance账户boshu_bntest13@dftstat.com的现货账户提走 7万 USDT, 转入OKX账户okcfcf01的funding账户中。

从内部跨所账户组cfdc_dcpro7(bn_v5_pm_pro22 + okx_v5_test41)的Binance账户bn61btc@dftstat.com的现货账户提走 33万 USDT, 转入OKX账户okzyusdt06的funding账户中。

从内部跨所账户组cfdc_dcpro9(bn_v5_pm_pro24 + okx_v5_test43)的Binance账户boshu_bncf01@dftstat.com的现货账户提走 30万 USDT, 转入OKX账户okzyusdt08的funding账户中。

从内部跨所账户组cfdc_dcpro16(bn_v5_pm_pro4 + okx_v5_test48)的Binance账户boshu_bntest20@dftstat.com的现货账户提走 41万 USDT, 转入OKX账户okzyusdt12的funding账户中。

从内部跨所账户组cfdc_dcpro17(bn_v5_pm_pro25 + okx_v5_test49)的Binance账户binancetfmmtest21@dftstat.com的现货账户提走 39万 USDT, 转入OKX账户okzyusdt13的funding账户中。

从内部跨所账户组cfdc_dcpro24(bn_v5_pm_pro19 + okx_v5_test22)的Binance账户binancetfmmtest35@dftstat.com的现货账户提走 6.16 BTC, 转入OKX账户okzybtc03的funding账户中。

从内部跨所账户组cfdc_dcpro29(bn_v5_pm_pro38 + okx_v5_test28)的Binance账户BINANCETFMMTEST9@dftstat.com的现货账户

In [10]:
all_data[all_data['类型'] =='托管'].sort_values(by = 'diff_ratio', ascending=False)

,跨所环境,本金,Binance本金,OKX本金,Binance环境,Binance账户名,OKX环境,OKX账户名,类型,okx_accountEquity,okx_uniMMR,binance_accountEquity,binance_uniMMR,total_equity,transfer_equity,diff_ratio,OKX_BTCUSDT_Price,OKX_ETHUSDT_Price,final_transfer,abs_diff_ratio
25,cfdc_dcpro28,ETH,200.00,400.00,bn_v5_pm_pro37,eth0110_virtual@bsdgwecjmanagedsub.com,okex5_v4_maker_cross_test8,eth0110,托管,519207.50,27.43,546136.55,33.71,1065344.05,-13464.53,-0.03,69285.90,2083.01,-6.46,0.03
13,cfdc_dcpro14,BTC,20.00,10.00,bn_v5_pm_pro17,bloqbtcrndlcxa_virtual@qsbr0fhxmanagedsub.com,okx_v5_test46,BLOQBTCRNDLCXA,托管,1000412.90,34.37,1143898.10,36.12,2144311.00,-71742.60,-0.07,69285.90,2083.01,-1.04,0.07
22,cfdc_dcpro25,ETH,200.00,321.74,bn_v5_pm_pro34,bncf01ran20251130_virtual@bjfkmrxumanagedsub.com,okx_v5_test31,cf01ran20241229,托管,495167.51,36.93,596214.70,43.46,1091382.20,-50523.59,-0.09,69285.90,2083.01,-24.26,0.09
20,cfdc_dcpro21,BTC,12.00,12.00,bn_v5_pm_pro29,sausdt0805_virtual@6ifg29vsmanagedsub.com,okx_v5_test20,ranxusdt0331,托管,748059.71,41.09,912993.08,86.86,1661052.79,-82466.69,-0.10,69285.90,2083.01,-1.19,0.10
30,cfdc_dcpro33,BTC,6.00,7.00,bn_v5_pm_pro41,btc0110_virtual@ddmo1lwgmanagedsub.com,okx_v5_test9,btc0110,托管,398909.05,20.91,498527.28,30.07,897436.32,-49809.12,-0.11,69285.90,2083.01,-0.72,0.11
19,cfdc_dcpro20,BTC,11.00,15.15,bn_v5_pm_pro28,fuxi1229_virtual@1ib0gs69managedsub.com,okx_v5_test11,okbtc0117,托管,786425.66,106.17,997772.75,65.32,1784198.41,-105673.54,-0.12,69285.90,2083.01,-1.53,0.12
28,cfdc_dcpro31,BTC,5.00,5.00,bn_v5_pm_pro39,bncfb720251209_virtual@wcr4jedjmanagedsub.com,okx_v5_test52,okxcfb720251209,托管,302142.03,32.23,393450.91,69.25,695592.94,-45654.44,-0.13,69285.90,2083.01,-0.66,0.13
29,cfdc_dcpro32,ETH,200.00,300.00,bn_v5_pm_pro40,sausdt1014_virtual@sopuwohkmanagedsub.com,okx_v5_test33,cfshi20250110,托管,456129.40,22.83,607877.96,43.24,1064007.36,-75874.28,-0.14,69285.90,2083.01,-36.43,0.14
34,cfdc_dcpro38,BTC,15.00,15.00,bn_v5_pm_pro46,bloqbtcrndlcxa2_virtual@v9h5b6ntmanagedsub.com,okx_v5_test55,BloqBTCRndlCXA2,托管,887295.30,22.91,1193115.71,54.96,2080411.01,-152910.20,-0.15,69285.90,2083.01,-2.21,0.15
18,cfdc_dcpro19,ETH,200.00,363.84,bn_v5_pm_pro2,digifinexbtc_virtual@o4ooi44pmanagedsub.com,okx_v5_test6,oketh0109,托管,441604.29,55.74,655382.07,23.37,1096986.36,-106888.89,-0.19,69285.90,2083.01,-51.31,0.19
